## 蛋白质分子动力学模拟

Author: Mingyuan Zhang
Date: 27/11/2023

使用分子动力学（Molecular Dynamics, MD）对蛋白质分子的动态过程进行计算机模拟如今已经成为蛋白质科学研究中的常规手段。本教程从实际应用的角度出发，仅使用python语言，在jupyter notebook中演示简单的计算机模拟（建模-->模拟-->分析）的全流程。

**本教程将使用以下软件包**：
1. OpenMM：MD模拟软件包，支持GPU加速。https://openmm.org/
2. nglview： 可视化软件，用于观察MD模拟轨迹。https://github.com/nglviewer/nglview
3. MDAnalysis： MD轨迹分析软件。https://www.mdanalysis.org/
4. numpy： 数据处理与计算。 https://numpy.org/
5. matplotlib： 数据可视化。 https://matplotlib.org/

**完成本教程的学习，你将会**：
1. 熟悉蛋白质体系MD模拟的全流程，使用OpenMM对蛋白质体系进行MD模拟。
2. 使用nglview对模拟轨迹进行观察，捕捉蛋白质折叠过程中的有趣现象。
3. 对观察到的现象使用MDAnalysis进行数值量化，并将量化结果作图进行可视化。

**模拟体系简介**：

考虑到对蛋白质体系进行MD模拟的庞大的计算量以及上手难度的降低，我们在这里使用一个简单的由10个丙氨酸(Ala)组成的短肽deca-alanine（ala10）作为模型系统进行展示。尽管模型非常简单，但是本教程中的全流程均适用于任意水溶液蛋白体系。在所有氨基酸中，丙氨酸相比其他氨基酸对形成$\alpha$螺旋（alpha helix）有强烈的倾向性。ala10作为短肽在水中没有固定构象，可以视为是一种内在无序蛋白（intrinsic disordered protein，IDP)，其在水溶液中会在完全的非折叠状态（unfolded state）和螺旋状态（helical state）之间快速切换，并在极其偶然的情况下会形成$\beta$发夹（beta hairpin）。我们将使用分子动力学模拟对这一蛋白折叠过程进行观察学习以及过程数值分析。

#### 导入需要使用的软件包

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from openmm.app import *
from openmm import *
from openmm.unit import *
from sys import stdout
import MDAnalysis as md
import nglview as nv
import numpy as np
from matplotlib import pyplot as plt
from nglview.datafiles import PDB, XTC

### 第1部分：分子动力学模拟体系建模与模拟参数设置
所有蛋白质水溶液MD模拟体系都需要以下输入才可以开始：
1. 蛋白质分子以及水分子的所有原子的初始坐标
2. 蛋白质分子的拓扑（原子类别、成键信息等）
3. 力场：决定体系中原子的势能方程，决定原子的运动方式
4. 模拟参数（见下）

**步骤1.1**： 使用OpenMM的PDBFile将pdb文件读取进OpenMM软件

In [ ]:
pdb = PDBFile('ala10.pdb')

In [ ]:
# 先用nglview看看ala10长什么样子，后面会详细介绍nglview的功能
u = md.Universe('ala10.pdb')
protein = u.select_atoms('protein')
view = nv.show_mdanalysis(protein)
view.clear_representations()
view.add_licorice('protein')
view.add_cartoon('protein')
view

In [ ]:
# 蛋白质分子总计103个原子的x，y，z坐标信息（103*3）被储存了下来
print(pdb.getPositions(asNumpy=True).shape)

In [ ]:
# OpenMM会根据pdb文件自动生成蛋白质分子的拓扑
print(pdb.topology)

In [ ]:
# 并且会记录每一个氨基酸和每一个原子（未展示）的名字
for residue in pdb.getTopology().residues():
    print(residue)

**步骤1.2：**选择MD模拟力场

In [ ]:
# 这里使用了OpenMM内置的主流蛋白质力场amber99sbildn以及主流水模型tip3p
forcefield = ForceField('amber99sbildn.xml','tip3p.xml')

**步骤1.3：**模拟体系建模：添加蛋白质缺失的氢原子，建立pbc边界条件（盒子）以及添加溶剂

**注意1.3.1**：这里仅调用了OpenMM内置的Modeller模块进行了建模操作，该模块仅支持处理少量简单的建模任务。在实际应用过程中，来自于晶体结构或冷冻电镜结构的pdb文件可能会出现缺失残基、整段loop区域或金属离子辅因子等情况。这种情况下需要借助其他软件来完成系统的建模，这里简单列举几种：

1.Modeller: https://salilab.org/modeller/

2.CHARMM-GUI: https://www.charmm-gui.org/

3.MMTSB: http://feig.bch.msu.edu/mmtsb/Main_Page

In [ ]:
# 调用Modeller模块，读取拓扑和坐标信息
model = Modeller(pdb.topology, pdb.positions)
# 根据模拟的pH,对缺失的氢原子进行补全（本例中pdb无氢原子缺失，仅作展示，无实际效果，因此会显示10个None,表示未添加）
model.addHydrogens(forcefield,pH=7.0)

In [ ]:
# 建立pbc边界性条件（盒子），添加溶液，添加0.15M浓度的离子，并使体系不存在净电荷
model.addSolvent(forcefield,model='tip3p',padding=1.5*nanometer,neutralize=True,positiveIon='Na+', negativeIon='Cl-', 
                 ionicStrength=Quantity(0.15*molar))

**注意1.3.2**：OpenMM中的pbc盒子以及溶液和离子的添加均在这一步通过model.addSolvent()完成，这里简单介绍部分可能歧义选项的含义：

1. model:添加的溶液的水模型的名称
2. padding：建立pbc边界性条件，使盒子的边缘距离蛋白质的任意原子不得小于1.5nm
3. neutralize：添加离子使体系中和，不存在净电荷

详情见OpenMM python API页：http://docs.openmm.org/latest/api-python/generated/openmm.app.modeller.Modeller.html?highlight=addsolvent#openmm.app.modeller.Modeller.addSolvent

**注意1.3.3**：在OpenMM中，任何有单位的数值在使用 from openmm.unit import * 后均可以以数值*单位名称或数值/单位名称的方式表示

如1.5*nanometer代表1.5nm，1/picosecond代表1ps$^{-1}$

In [ ]:
# 将建模完成后的系统的坐标和拓扑导入pdb这个python object中
pdb.positions = model.getPositions()
pdb.topology = model.getTopology()

In [ ]:
# 经过Modeller处理后的拓扑，可以看到增加了许多新的原子（水）
pdb.topology

In [ ]:
# 将建立好的模型保存
with open('withsolvent.pdb','w+') as f:
    pdb.writeFile(pdb.topology,pdb.positions,f)

In [ ]:
# 也可以用nglview可视化检查一下模型(后面会介绍nglview的使用)
u = md.Universe('withsolvent.pdb')
all = u.select_atoms('not name P')
view = nv.show_mdanalysis(all)
view.clear_representations()
view.add_cartoon('protein',color='blue')
view.add_licorice('water')
view.add_hyperball('ion',aspectRatio=1)
view

**步骤1.4**：添加模拟参数，创建模拟系统system，添加integrator

In [ ]:
# 由OpenMM的forcefield object创建系统，将由Modeller建立的模型中的拓扑以及一系列MD模拟参数打包生成一个叫system的object
system = forcefield.createSystem(pdb.topology, nonbondedMethod=PME,nonbondedCutoff=1.2*nanometer, 
                                 constraints=AllBonds)

**注意1.4.1：** 这里没有进一步说明各模拟参数的意义是因为背后的理论知识非常复杂（以下同理），有兴趣可以自行查阅OpenMM文档或google

In [ ]:
# 分子动力学每一步分子动态的算法，称之为integrator，设置步长为2fs（全原子蛋白质体系常用步长）。添加NVT温控，体系温度为300K。
integrator = NoseHooverIntegrator(300*kelvin,1/picosecond,0.002*picosecond)

**注意1.4.2：** 这里采用的NoseHooverIntegrator，是在integrator中本身整合了温控（thermostat）的功能，因此系统无需在后续NVT平衡前再额外添加温控。使用此integrator的模拟本身就已经是在对NVT系综进行采样。

**步骤1.5**：创建模拟，导入初始坐标，完成模型与MD系统搭建

In [ ]:
# 将模型的topology，系统的所有参数信息system，以及integrator全部整合在一起，得到最终准备好进行MD模拟的simulation object。
simulation = Simulation(pdb.topology, system, integrator)

# 将Modeller建模的坐标信息导入simulation
simulation.context.setPositions(pdb.positions)

### 第2部分：使用OpenMM进行分子动力学模拟的计算
蛋白质体系模拟主要分为以下几个步骤：
1. 能量最小化（Energy Minimization）：使用gradient descent算法对系统的能量进行快速降低，防止因建模存在的不合理性（如原子重叠）导致的系统崩溃
2. NVT平衡（NVT Equilibration）：将系统温度稳定在NVT系综指定的温度下
3. NPT平衡（NPT Equilibration）：添加控压（barostat）使系统压强稳定在NPT系综指定的压强下
4. 正式MD模拟计算 （Production）：最终用于分析ala10折叠的轨迹

**步骤2.1：**能量最小化

In [ ]:
# 能量最小化
energy_before = simulation.context.getState(getEnergy=True)
simulation.minimizeEnergy()
energy_after = simulation.context.getState(getEnergy=True)

In [ ]:
# 对比能量最小化前后的能量，可以发现系统的能量被大幅降低了
print('Before energy minimization:' + str(energy_before.getPotentialEnergy()) 
      + '\n' + 'After energy minimization:' + str(energy_after.getPotentialEnergy()))

**步骤2.2**：设置不同阶段的模拟时长

In [ ]:
# 设置模拟时长
nvt_length = 50000
npt_length = 50000
md_length = 100000
total_length = nvt_length + npt_length + md_length

**注意2.2.1：** 这里模拟时长的单位是步数，因此50000实际对应的模拟时长为 50000 * 2fs （之前设置的步长）= 0.1ns。一般水溶液中蛋白的NVT平衡时间为0.1ns，NPT平衡时间为0.2ns，只要温度和压强都稳定在设定值附近即可。正式模拟时间则需要视具体体系而定：使用MD观察你感兴趣的现象往往需要进行数倍甚至数十倍该现象现实世界所需时长的MD模拟。目前主流生物学期刊所进行的MD模拟累计时长一般在0.5-10$\mu$s之间。这里仅进行0.2ns的MD是受限于计算条件，仅用于展示流程。

**注意2.2.2：** 以上设置在我的笔记本cpu需要总计约8分钟左右完成，如果你的硬件在下一步模拟时显示无法在8分钟内完成，可以考虑缩短所有时长至最多1/5

**步骤2.3：**NVT平衡

In [ ]:
# 清理掉所有Reporters（见下），防止重复运行导致的错误
simulation.reporters.clear()

# 添加温控：由于integrator本身已经内置温控功能，无需添加。如需添加可使用以下两行命令：
# system.addForce(AndersenThermostat(300*kelvin, 1/picosecond))
# simulation.context.reinitialize(preserveState=True)

### 为simulation添加reporters用于实时输出

# DCDReporter输出系统中所有坐标信息至DCD文件nvt.dcd，用于后续分析
simulation.reporters.append(DCDReporter('nvt.dcd', 1000))
# StateDataReporter输出系统目前所有的状态信息以及模拟进度，速度和剩余时间等，stdout表示在jupyter notebook中即时输出。
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,potentialEnergy=True, totalEnergy=True,temperature=True, 
                            progress=True,remainingTime=True, speed=True, volume=True, totalSteps=total_length,separator='\t'))
# 同时将所有状态信息输出至硬盘中
simulation.reporters.append(StateDataReporter('nvt.log', 1000, step=True,potentialEnergy=True, totalEnergy=True,temperature=True, 
                            progress=True,remainingTime=True, speed=True, volume=True, totalSteps=total_length,separator='\t'))

# 开始NVT平衡
simulation.step(nvt_length)

**观察2.3：** 

1. 系统温度是如何变化的？控温是否让温度稳定在了设定值附近？
2. 盒子体积是否发生了变化？

**步骤2.4：** 建立NVT存档点

In [ ]:
# 恭喜你完成了MD模拟计算部分的第1/3步，留下一个存档点吧
simulation.saveCheckpoint('nvt.chk')
# 清理掉所有Reporters，防止重复运行导致的错误
simulation.reporters.clear()

**注意2.4：** 读取存档

如果你的模拟因为任何原因需要从NPT之前重新开始，重新运行除步骤2.3：NVT平衡和步骤2.4：建立NVT存档点以外所有的代码块，再运行下方的代码块即可继续之后的步骤。

In [ ]:
# Load checkpoint
#simulation.loadCheckpoint('nvt.chk')
#simulation.reporters.clear()

**步骤2.5：** NPT平衡

In [ ]:
# 清理掉所有Reporters，防止重复运行导致的错误
simulation.reporters.clear()

# 添加控压，设置压强为1个大气压
system.addForce(MonteCarloBarostat(1*bar, 300*kelvin))
# 在OpenMM中，对system使用addForce添加了新的Force之后，必须对simulation进行reinitialize才会对正在进行的模拟进行更新，
# 否则simulation并不会因为system的改变而改变，注意使用preserveState=True保留当前所有的模拟状态信息。
simulation.context.reinitialize(preserveState=True)

# 添加Reporters,同NVT
simulation.reporters.append(DCDReporter('npt.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,potentialEnergy=True, totalEnergy=True,temperature=True, 
                            progress=True,remainingTime=True, speed=True,volume=True,totalSteps=total_length,separator='\t'))
simulation.reporters.append(StateDataReporter('npt.log', 1000, step=True,potentialEnergy=True, totalEnergy=True,temperature=True, 
                            progress=True,remainingTime=True, speed=True,volume=True,totalSteps=total_length,separator='\t'))

# 开始NPT平衡
simulation.step(npt_length)

**观察2.5：**  盒子体积是否发生了变化？为什么在NPT平衡的过程中出现了这种状况，而NVT平衡的过程中没有？

**注意2.5：** OpenMM中的Reporters功能并不支持计算、输出压强，因此我们只能默认通过足够步数的平衡，压强已经稳定在我们的设定值附近

**步骤2.6：** 建立NPT存档点

In [ ]:
# 恭喜你完成了MD模拟计算部分的第2/3步，留下一个存档点吧
simulation.saveCheckpoint('npt.chk')
# 清理掉所有Reporters，防止重复运行导致的错误
simulation.reporters.clear()

**注意2.6：** 读取存档

如果你的模拟因为任何原因需要从正式MD之前重新开始，重新运行除步骤2.3、步骤2.4、步骤2.5、步骤2.6以外所有的代码块，再运行下方的代码块即可继续之后的步骤。

In [ ]:
# Load checkpoint
#simulation.loadCheckpoint('npt.chk')
#simulation.reporters.clear()

**步骤2.7：** 正式MD模拟

In [ ]:
# 清理掉所有Reporters，防止重复运行导致的错误
simulation.reporters.clear()

# 添加Reporters,同NVT,NPT
simulation.reporters.append(DCDReporter('md.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,potentialEnergy=True, totalEnergy=True,temperature=True, 
                            progress=True,remainingTime=True, speed=True,volume=True,totalSteps=total_length,separator='\t'))
simulation.reporters.append(StateDataReporter('md.log', 1000, step=True,potentialEnergy=True, totalEnergy=True,temperature=True, 
                            progress=True,remainingTime=True, speed=True,volume=True,totalSteps=total_length,separator='\t'))

# MD,启动！
simulation.step(md_length)

**步骤2.8（选做）：** 建立正式MD存档点，如果你还想之后找时间续跑的话

In [ ]:
# 恭喜你完成了MD模拟计算的完整三部曲！
simulation.saveCheckpoint('md.chk')
# 清理掉所有Reporters，防止重复运行导致的错误
simulation.reporters.clear()

**注意2.8（选做）**：如果你想要继续续跑此轨迹，请注意调整reporters中totalSteps这一项，使之与你计划模拟的总时长（包含NVT，NPT以及你已经跑的正式MD的时长）相符，否则reporters中输出的百分比进度条和剩余时间将会发生错乱。

### 第3部分：观察轨迹并使用MDAnalysis进行量化分析

考虑到时间有限，这里我提供一段提前跑好的ala10 5$\mu$s 5000帧的轨迹，尽管少数参数设置存在一定差异性，但可以看作是以上正式MD的完整数据。学会如何分析MD轨迹，从超高时间和空间分辨率的高维数据中提取并量化具有生物物理意义的关键信息，才是掌握MD这项技术的重点。这里我们以ala10折叠成$\alpha$螺旋作为例子，展示如何使用MDAnalysis提取出蛋白质在MD轨迹中二级结构变化的信息。

以下分析方法均可直接应用到实际科研实践中，适用于任意蛋白质体系。

**步骤3.1：** 导入轨迹与分子拓扑文件。与OpenMM的PDBFile模块类似，MDAnalysis将所有信息存储在Universe中

In [ ]:
# 创建MDAnalysis Universe,导入拓扑以及轨迹
u = md.Universe('analysis/ala10.pdb','analysis/ala10.xtc')

**注意3.1：** 这里提供的轨迹是经过后处理，去除了溶剂并且修复了pbc的轨迹的来自于另一个分子动力学Gromacs的xtc轨迹文件。由于我们这里的分析只涉及蛋白质分子内部之间的相互作用，所以去除溶剂可以有效节省硬盘以及内存空间。实际应用中，请根据自己的实际需求进行相应调整。

In [ ]:
# 与PDBFile类似，Universe中记录了残基以及原子的信息
u.residues,u.atoms

**步骤3.2：** 使用nglview对轨迹进行观察

In [ ]:
# 使用MDAnalysis选择整个蛋白质分子
protein = u.select_atoms('protein')
# 创建nglview object
view = nv.show_mdanalysis(protein)

In [ ]:
# 观察轨迹
view

**注意3.2.1：**

这里简单介绍nglview的使用：

1. 鼠标悬置于视窗内，左下角的三个按钮分别代表播放、暂停以及视角锁定。
2. 可以拖动进度条观察轨迹，也可以点击进度条旁边的数字输入想观察的帧数。
3. 右上角的按钮点击后可以全屏。
4. 鼠标左键和滚轮可以控制视角角度以及缩放。
5. 鼠标右键点击一次第一个原子，然后连续鼠标右键点击两次第二个原子，可以在轨迹上标记两个原子之间的距离。
6. 可以使用下面的指令切换不同的表示形式。

其他更高级的操作可以参考：https://github.com/nglviewer/nglview 中的example

In [ ]:
# Clear all representations to try new ones
view.clear_representations()
# add licorice without hydrogen
view.add_licorice('protein')

**观察3.2：**

根据实际需要，运用nglview的各种功能调整ala10的显示方式，观察：
1. ala10在5$\mu$s的过程中多次形成$\alpha$螺旋，你可以通过观察轨迹找到多少个这样的构象？
2. ala10在过程中是否有形成过$\beta$-hairpin？

**注意3.2.2：** 量化观察结果

我们通过观察轨迹所得到的结论，是我们个人的**主观**描述。这种主观描述在不同的人之间必然存在偏差，因此我们需要引入以下方法对我们观察到的现象进行基于数值量化的（相对）**客观**的论述与佐证。下面，我们从$\alpha$螺旋的定义出发，用两种不同的数值量化方法，（相对）**客观**地描述轨迹中$\alpha$螺旋的形成。

**步骤3.3：** 量化分析——主链氢键

这里我们从这里这里我们从蛋白质二级结构的定义出发，分析蛋白质主链的氮和氧原子（如果严格一点应该使用i和i+4的定义，但是这里选择放松标准，为了将更多的中间态以及可能存在的$\beta$二级结构描述进来）是否形成氢键——而这需要我们首先定义什么是氢键。

这里我们使用了过去常用的经验值：即连接氮原子的氢原子（HN）与氧原子（O）之间距离小于2.5Angstrom,我们即视为形成氢键，并在此基础上排除掉所有相邻氨基酸的HN-O配对。将这个标准利用MDAnalysis的distance_array功能，写成函数backbone_hbond，对每一个给定构象进行氢键数量的计数。

In [ ]:
from MDAnalysis.analysis import distances
### 氢键计数
def backbone_hbond(universe,distance_cutoff=2.5):   # 设定筛选距离为2.5Angstrom
    # 选定构建distance_array的原子
    donor = u.select_atoms('name HN')
    acceptor = u.select_atoms('name O')
    # 利用distance_array函数构建2d array,array中记录每一个HN-O原子对之间的距离，最终形成9*9的distance_array
    distance_array = distances.distance_array(donor.positions,acceptor.positions)
    # 筛选符合条件的HN-O原子对
    pairs = np.array(np.where(distance_array < distance_cutoff)).T    
    hbond_count = 0
    # 去掉所有相邻氨基酸的原子对，并计数
    for pair in pairs:
        if abs(pair[0]-pair[1]) != 1:
            hbond_count += 1
    return hbond_count

构建好函数之后，我们对整条轨迹的每一帧进行氢键数量的统计，并使用matplotlib绘图。

In [ ]:
# Universe.trajectory中记录了由xtc导入的每一帧的信息，使用简单的for loop对整条轨迹进行氢键数量的计算
hbond_traj = []
for ts in u.trajectory:
    count_ts = backbone_hbond(u)
    hbond_traj.append(count_ts)

# matplotlib绘图：轨迹中随时间变化的氢键数量，代表了ala10的折叠与解折叠
plt.plot(hbond_traj)
plt.xlim(0,5000)
plt.ylim(0,9)
plt.xlabel('Time (ns)')
plt.ylabel('Number of Hydrogen Bonds')
plt.title('Number of Hydrogen Bonds over 5000 ns MD simulation')
plt.show()

**观察3.3.1**：我们可以从图中看到，整条轨迹在氢键数量0-8之间快速切换，只有少数构象拥有较多的氢键数量。
使用以下命令将这些构象提取出来进行可视化，我们使用经验距离量化氢键数量进而描述二级结构的方法成功了吗？

In [ ]:
# 将氢键大于一定数量的构象提取并保存至pdb文件，使用nglview进行可视化
hbond_idx = np.where(np.array(hbond_traj)> 5)   # 在这里通过修改代码来更改筛选指标
protein.write('hbond.pdb', frames=list(hbond_idx[0]))
print('我们总计提取了{len}帧符合指标的构象'.format(len=len(list(hbond_idx[0]))))
hbond_u = md.Universe('hbond.pdb')
hbond_view_protein = hbond_u.select_atoms('protein')
hbond_view = nv.show_mdanalysis(hbond_view_protein)

In [ ]:
# 观察提取的构象
hbond_view

**观察3.3.2**：通过提高和降低上面代码中氢键数量的筛选指标，我们可以得到不同数量的构象。
将筛选指标任意更改（如改为等于特定数量，小于特定数量）对比不同指标得到的构象，随着我们定义的函数的输出数值的改变，我们是否可以大致描述ala10的折叠过程？

**步骤3.4：** 量化分析——二面角

从二级结构的特性出发：由ramachandran plot可知每一种二级结构有其对应的特定的二面角范围。也就是说，通过计算蛋白的二面角我们也可以一定程度地量化蛋白的二级结构。
考虑到我们这里每一个单个的氨基酸所处的化学环境是基本一致的，我们首先将整条轨迹每一帧蛋白分子所有的二面角投影到二面角的平面，从二面角的角度来看看ala10在模拟过程中可能存在什么样的二级结构。

我们使用MDAnalysis自带的Ramachandran函数对所有非两端氨基酸进行二面角的分析。

In [ ]:
# 选择非两端氨基酸进行Ramachandran分析
from MDAnalysis.analysis.dihedrals import Ramachandran
res = u.select_atoms("resid 2-9")
dihedrals = Ramachandran(res).run()

In [ ]:
# 对整条轨迹所有采集到的氨基酸投射到二面角平面，绘图分析
fig, ax = plt.subplots(figsize=(6,6))
plot = dihedrals.plot(ax=ax, color='k',ref=False)

可以看到，我们的数据点非常多（5000*8=40000个点），并且均匀覆盖在了几乎所有二面角可以到达的区域。
从这张图里，我们可以知道ala10的不同氨基酸在不同的时间点会存在非常多二级结构的可能性，但我们并不知道他随着时间折叠或解折叠的过程，
也无法使用上面统计氢键的方法将折叠、中间态、非折叠的构象提取出来。

**步骤3.5：** 量化分析——基于二面角的函数helicity

使用二面角的信息描述整个蛋白的二级结构，比如说形成螺旋的程度，我们必须使用一个特殊的函数对角度进行特定的转换并将每一个氨基酸的函数值进行相加。
这个转换同时还必须选择一个对应特定二级结构的二面角参考值，这样当二面角接近这个参考值时，我们才可以通过这个函数说明，这里的二面角信息使他接近了比如说$\alpha$螺旋二面角的值，从而将信息转换成了“这里形成了$\alpha$螺旋”。

我们这里采样了这样一个特定的转换函数，来自软件PLUMED的ALPHABETA函数 https://www.plumed.org/doc-v2.9/user-doc/html/_a_l_p_h_a_b_e_t_a.html：

$$
s = \frac{1}{2}\sum_i [1+\cos(\phi_i-\phi_i^{ref})]
$$

设置$\phi_i^{ref}=-45\degree$并计算所有区域内$\phi,\psi$角时，这个函数被广泛用于描述蛋白质折叠过程中该区域$\alpha$螺旋的形成（因为当phi和psi角均为$-45\degree$时，其在phi/psi平面上处于$\alpha$螺旋的区域）。当第i个二面角偏离$\phi_i^{ref}$时，加和符号内的值会逐渐趋近于0,反之则会趋近于1。因此，当最后这个加和数值越大时，表示当前蛋白的二面角接近于$\alpha$螺旋的二面角，因此可以推测该蛋白分子现在形成了$\alpha$螺旋的结构。

这里我们将这个函数在python里面重写了一遍，直接嵌入在了for循环里进行每一帧的计算。我们将这个函数称为helicity。

In [ ]:
# 从MDAnalysis的Ramachandran函数里提取出所有二面角的数据
dihedrals_angles = dihedrals.angles
# for循环计算每一帧的helicity
helicity_traj = []
for dihedral_t in dihedrals_angles:
    # 将角度转换为rad单位
    dihedrals_rad = dihedral_t/180*3.1416
    # 定义helicity函数
    helicity = 0.5*np.sum(np.cos(dihedrals_rad.T--0.7855)+1)
    helicity_traj.append(helicity)

In [ ]:
# 将helicity随时间变化的轨迹绘图
plt.plot(np.arange(0,5001,10),helicity_traj[::10])
plt.xlim(0,5000)
plt.ylim(4,16)
plt.xlabel('Time (ns)')
plt.ylabel('Helicity')
plt.title('Helicity over 5000 ns MD simulation')

**观察3.5：** 

与基于氢键的分析类似，思考如何设置合理的筛选值，用下面同样的流程对构象进行筛选，并进行可视化验证，这个函数可以达到同样的目的吗？

和直接统计氢键的数量相比，这个算法有什么好处？

In [ ]:
# 将helicity大于一定数值的构象提取并保存至pdb文件，使用nglview进行可视化
helicity_idx = np.where(np.array(helicity_traj)> 14) # 在这里通过修改代码来更改筛选指标
protein.write('helicity.pdb', frames=list(helicity_idx[0]))
print('我们总计提取了{len}帧符合指标的构象'.format(len=len(list(helicity_idx[0]))))
helicity_u = md.Universe('helicity.pdb')
helicity_view_protein = helicity_u.select_atoms('protein')
helicity_view = nv.show_mdanalysis(helicity_view_protein)

In [ ]:
# 观察提取的构象
helicity_view